In [269]:
## Requirements, uncomment first run through ## 

# %pip install nest_asyncio
# %pip install playwright
# %pip install beautifulsoup4
# %pip install letterboxdpy
# %pip install pandas 
# % pip install lxml
# !python -m playwright install

## End Requirements

import pandas as pd
import re
import os
from bs4 import BeautifulSoup
from letterboxdpy.movie import Movie
from datetime import datetime


In [270]:
## run this code to install playwright broswer binaries

# !python -m playwright install

In [271]:
import os 

def find_repo_root(start_path):
    """
    useful general function for finding the (first, closest) repo root so github file paths work the same on different machines 
    """
    current_path = os.path.abspath(start_path)
    
    while True:
        # Check for the existence of the .git directory or other indicators
        if os.path.isdir(os.path.join(current_path, '.git')) or \
           os.path.isfile(os.path.join(current_path, 'README.md')):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        
        # Stop if we reach the root directory
        if parent_path == current_path:
            break
        
        current_path = parent_path

    return None  # Return None if not found

root = find_repo_root(os.getcwd())
root = root.replace("\\", "/")
root

'c:/Users/fitsl/Documents/Programming/UVM Programming Classes/PoCS/pocs_project'

In [272]:

movie_instance = Movie("Clothes-of-deception")
movie_instance.year
movie_instance.url

'https://letterboxd.com/film/clothes-of-deception/'

In [273]:
# ## Replace the filepath with wherever you store the 2020 update trope dataset 

# df = pd.read_csv(f"{root}/Data/2020_trope_data/2020_updated_tropeset.csv") 
# print(df.shape)
# df.head()

In [274]:
# df = df[['NameIMDB', 'Year', 'Rating', 'Id']]

# df['letterboxd_search'] = "https://letterboxd.com/search/" + df['NameIMDB'] + " " + df['Year'].astype(str) + '/'
# df['letterboxd_search'] = df['letterboxd_search'].map(
#     lambda x: str(x).replace(" ", "+"))
# print(df.shape)
# df.head()
# df.to_csv(f"{root}/Data/2020_trope_data/letterboxd_search.csv")

In [275]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.options import Options
# import time
# from datetime import datetime


# def get_Movies_all(df, chunk_size=50, begin=0, length=None, 
#                    output_file=f'{root}/Data/tropes_year_movie.csv',
#                    sleeptime=1):
#     # Set up Chrome options
#     chrome_options = Options()
#     chrome_options.add_argument("--headless")  

#     if not length:
#         length =len(df)

#     # Automatically manage ChromeDriver
#     service = Service(ChromeDriverManager().install())
#     driver = webdriver.Chrome(service=service, options=chrome_options)

#     all_movies = []  # To collect all movie results

#     # Process the DataFrame in chunks
#     for start in range(begin, length, chunk_size):
#         current_time = datetime.now()
#         print(f"started {start} to {start + chunk_size}  at  " + current_time.strftime("%Y-%m-%d %H:%M:%S"))

#         chunk = df.iloc[start : start + chunk_size].copy()
#         urls = chunk['letterboxd_search']

#         for i, url in enumerate(urls):
#             time.sleep(sleeptime)  # Time to wait between requests (if any)
#             try:
#                 driver.get(url)
#             except Exception as e:
#                 print(f"error with {url}: {e}")

#             html_source = driver.page_source
#             soup = BeautifulSoup(html_source, 'html.parser')

#             # Get elements from soup and append them into the list
#             try:
#                 # first_instance = soup.find('a', class_='frame', href=lambda href: href and href.startswith('/film/'))   first option
#                 first_instance = soup.find('a', href=lambda href: href and href.startswith('/film/'))
#                 if first_instance:
#                     title = first_instance['href'].rstrip('/').split('/')[-1]
#                     movie = Movie(title)
#                     all_movies.append((url, movie))
#                 else:
#                     all_movies.append((url, None))
#                     print(f"Line {i} in block. No results on page for {url}. Try other year maybe.")
#             except Exception as e:
#                 all_movies.append((url, None))
#                 print(f"{url} not found. Try other year maybe. Error: {e} Line{i}")

#             # Save to CSV every 100 results processed
#             if len(all_movies) % chunk_size == 0:
#                 # Create a temporary DataFrame and append to CSV
#                 temp_df = pd.DataFrame(all_movies, columns=['letterboxd_search', 'Movie'])
#                 temp_df.to_csv(output_file, mode='a', header=not pd.io.common.file_exists(output_file), index=False)
#                 all_movies = []  # Reset the list after saving
#                 print(f"finished {start} to {start + chunk_size}  at  " + current_time.strftime("%Y-%m-%d %H:%M:%S"))



#     # Save any remaining results
#     if all_movies:
#         temp_df = pd.DataFrame(all_movies, columns=['letterboxd_search', 'Movie'])
#         temp_df.to_csv(output_file, mode='a', header=not pd.io.common.file_exists(output_file), index=False)

#     # Clean up
#     driver.quit()
#     return df


In [276]:
df = pd.read_csv(f"{root}/Data/2020_trope_data/HIT_letterboxd_link_movies.csv")


In [ ]:
import ast

def fk_apply_literal(x):
    try:
        return ast.literal_eval(x)
    except Exception as e:
        # print(e, x)
        return None

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def get_parsed_page(url: str) -> None:
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://letterboxd.com",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")

def fk_movie_popular_reviews(movie, n=None) -> dict:
    try:
        base = movie.get('url')
    except Exception as e:
        print(e, movie)
        return None
    urls = []
    urls.append(base + "/reviews/by/activity")
    if n:
        for i in range(2, n+1): 
            urls.append(base + f"/reviews/by/activity/page/{i}")
   

    ret = []
    for url in urls:
        page = get_parsed_page(url) 


        film_details = page.find_all(class_='film-detail')
        for detail in film_details:
            curr = {}
            try:
                curr['stars'] = detail.select_one('.rating').get_text(strip=True)
            except:
                curr['stars'] = None

            try: 
                curr['review'] = detail.select_one('.body-text').get_text(strip=True)
            except:
                curr['review'] = None

            try:
                curr['date'] = detail.select_one('.date ._nobr').get_text(strip=True)
            except:
                curr['review'] = None
            ret.append(curr)
    return ret

In [278]:
def get_budget(movie, n=None) -> dict:
    try:
        url = movie.get('url')
    except Exception as e:
        print(e, movie)
        return None
    match = re.search(r"https?://(?:www\.)?letterboxd\.com/film/([^/]+)/?", url)
    match = (match.group(1))
    url =f'https://letterboxd.com/whatsthebudget/film/{match}'
    page = get_parsed_page(url) 


    budget_meta = page.find('meta', {'name': 'description'})  # or {'property': 'og:description'}
    if budget_meta:
        budget = budget_meta['content']
        return budget
    else:
        return pd.na


In [279]:
laura = Movie("laura")

In [280]:
get_budget(fk_apply_literal(
    laura
))

malformed node or string: <letterboxdpy.movie.Movie object at 0x000001E9619DA850> {
    "title": "laura",
    "url": "https://letterboxd.com/film/laura/",
    "directors": [
        "Otto Preminger"
    ],
    "rating": "4.04 out of 5",
    "year": "1944",
    "genres": [
        "Mystery",
        "Drama"
    ]
}
'NoneType' object has no attribute 'get' None


In [ ]:
def get_reviews_all(data, chunksize=50, begin=0, length=None, n=8, output_file=None): ## change the save name

    # first clean the frame
    df = data.copy()
    df['Movie'] = df['Movie'].apply(lambda x : fk_apply_literal(x))

    if not output_file:
        output_file = f'{root}/Data/2020_trope_data/Comments/movie_n={n}_comments.csv'

    if not length:
        length = len(df)

    for start in range(begin, length, chunksize):
        #print start time
        current_time = datetime.now()
        print(f"started {start} to {start + chunksize}  at  " + current_time.strftime("%Y-%m-%d %H:%M:%S"))

        # actual code 
        if length > start + chunksize:
            chunk = df.iloc[start : start + chunksize].copy()
        else:
            chunk = df.iloc[start: length].copy()
        
        chunk['comments'] = chunk['Movie'].apply(lambda x : fk_movie_popular_reviews(x, n))
        chunk['Budget'] = chunk['Movie'].apply(get_budget)
        print(chunk.iloc[0])

        chunk.to_csv(output_file, mode='a', header=not pd.io.common.file_exists(output_file), index=False)

        # print end time
        current_time = datetime.now()
        print(f"finished {start} to {start + chunksize}  at  " + current_time.strftime("%Y-%m-%d %H:%M:%S"))


In [286]:
get_reviews_all(df, n=10)

malformed node or string on line 6: <ast.Name object at 0x000001EA322D9390> {
    "title": "gorguts-live-at-the-whisky-a-go-go-1998",
    "url": "https://letterboxd.com/film/gorguts-live-at-the-whisky-a-go-go-1998/",
    "directors": [],
    "rating": "None found",
    "year": null,
    "genres": []
}
malformed node or string on line 6: <ast.Name object at 0x000001E9748DA590> {
    "title": "memorial-day-2000",
    "url": "https://letterboxd.com/film/memorial-day-2000/",
    "directors": [],
    "rating": "None found",
    "year": null,
    "genres": [
        "Documentary"
    ]
}
malformed node or string on line 6: <ast.Name object at 0x000001EA2CF3DC90> {
    "title": "rhapsody-live-at-rock-festival-2017",
    "url": "https://letterboxd.com/film/rhapsody-live-at-rock-festival-2017/",
    "directors": [],
    "rating": "None found",
    "year": null,
    "genres": []
}
started 0 to 50  at  2024-11-24 13:06:25
                                   letterboxd_search  \
0  https://letterbo

In [287]:
budgets =pd.read_csv(f"{root}/Data/2020_trope_data/Comments/movie_n=1_comments.csv")

In [288]:
budgets

,letterboxd_search,Movie,comments,Budget
0,https://letterboxd.com/search/ABCs+of+Death+2+...,"{'title': 'abcs-of-death-2', 'url': 'https://l...","[{'stars': '★★★', 'review': 'To the creators o...","BUDGET; UNKNOWN Gross worldwide $7,171"
1,https://letterboxd.com/search/ABCs+of+Death+2+...,"{'title': 'abcs-of-death-2', 'url': 'https://l...","[{'stars': '★★★', 'review': 'To the creators o...","BUDGET; UNKNOWN Gross worldwide $7,171"
2,https://letterboxd.com/search/ABCs+of+Death+2+...,"{'title': 'abcs-of-death-2', 'url': 'https://l...","[{'stars': '★★★', 'review': 'To the creators o...","BUDGET; UNKNOWN Gross worldwide $7,171"
3,https://letterboxd.com/search/A+Beautiful+Day+...,{'title': 'a-beautiful-day-in-the-neighborhood...,"[{'stars': '★★★', 'review': 'therapy: expensiv...","BUDGET: $25,000,000 / €23,011,567 BOX OFFICE (..."
4,https://letterboxd.com/search/A+Beautiful+Mind...,"{'title': 'a-beautiful-mind', 'url': 'https://...","[{'stars': '★★★★', 'review': 'ugh, his mind', ...","BUDGET: $58,000,000 / 53,243,768 EUR BOX OFFIC..."


In [289]:
get_budget(Movie("Aliens"))

'Movie' object has no attribute 'get' {
    "title": "aliens",
    "url": "https://letterboxd.com/film/aliens/",
    "directors": [
        "James Cameron"
    ],
    "rating": "4.12 out of 5",
    "year": "1986",
    "genres": [
        "Science Fiction",
        "Horror",
        "Thriller",
        "Action"
    ]
}


In [291]:
df = pd.read_csv(f"{root}/Data/2020_trope_data/Comments/movie_n=10_comments.csv")
df

,letterboxd_search,Movie,comments,Budget
0,https://letterboxd.com/search/ABCs+of+Death+2+...,"{'title': 'abcs-of-death-2', 'url': 'https://l...","[{'stars': '½', 'review': 'I couldn’t help but...",NaN
1,https://letterboxd.com/search/A+Beautiful+Day+...,{'title': 'a-beautiful-day-in-the-neighborhood...,"[{'stars': '★★★½', 'review': 'This movie has b...","BUDGET: $17,000,000 / €15,849,931"
2,https://letterboxd.com/search/A+Beautiful+Mind...,"{'title': 'a-beautiful-mind', 'url': 'https://...","[{'stars': '★★★★', 'review': 'Corman thought h...",NaN
3,https://letterboxd.com/search/A+Better+Tomorro...,"{'title': 'a-better-tomorrow', 'url': 'https:/...","[{'stars': '★★★★', 'review': 'Nobody understan...","BUDGET; UNKNOWN Gross worldwide $7,171"
4,https://letterboxd.com/search/A+Birder's+Guide...,"{'title': 'a-birders-guide-to-everything', 'ur...","[{'stars': '★★★', 'review': 'To the creators o...",NaN
...,...,...,...,...
95,https://letterboxd.com/search/A+Million+Ways+t...,"{'title': 'a-million-ways-to-die-in-the-west',...","[{'stars': '★★★', 'review': 'Weird GUARDIANS O...",NaN
96,https://letterboxd.com/search/A+Monster+Calls+...,"{'title': 'a-monster-calls', 'url': 'https://l...","[{'stars': '★★★½', 'review': 'Fred Willard cou...","BUDGET: $16,000,000"
97,https://letterboxd.com/search/A+Most+Violent+Y...,"{'title': 'a-most-violent-year', 'url': 'https...","[{'stars': None, 'review': 'From one angle, Co...","BUDGET: $40,000,000 / €36,791,071 BOX OFFICE (..."
98,https://letterboxd.com/search/A+Movie+1958/,"{'title': 'a-movie', 'url': 'https://letterbox...","[{'stars': '★★★', 'review': ""One of the most p...","BUDGET: $43,000,000 / €39,547,747 BOX OFFICE (..."
